# 选取数据源

In [1]:
# 首先导入使用的包，使用request进行网页请求，使用pandas保存数据。
import requests
import pandas as pd
import time
pd.set_option('max_rows',500)    #修改显示的最大行数，如果超过设置的行和列就显示省略号，

# 设置请求头，伪装为浏览器
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
}

# 发起请求，将找到的第一个数据源作为请求目标。
url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-total'   # 定义要访问的地址
r = requests.get(url, headers=headers)  # 使用requests发起请求
print(r.status_code)  # 查看请求状态，响应状态为200表示请求成功
print(type(r.text))    #查看类型。 r.text——文本获取。 r.content——图片、音频获取。
print(len(r.text))    # 查看文本长度

200
<class 'str'>
375715


In [2]:
# 将字符串格式转为json格式。
import json
data_json = json.loads(r.text)
data_json.keys()    # data键存放着需要的数据

dict_keys(['reqId', 'code', 'msg', 'data', 'timestamp'])

In [3]:
data = data_json['data'] # 取出json中的数据
data.keys()  

dict_keys(['chinaTotal', 'chinaDayList', 'lastUpdateTime', 'overseaLastUpdateTime', 'areaTree'])

# 世界各国实时数据获取 today_data

In [4]:
# 将提取数据的方法封装为函数
def get_data(data,info_list):
    info = pd.DataFrame(data)[info_list] # 主要信息
    
    today_data = pd.DataFrame([i['today'] for i in data ]) # 生成today的数据
    today_data.columns = ['today_'+i for i in today_data.columns] # 修改列名
    
    total_data = pd.DataFrame([i['total'] for i in data ]) # 生成total的数据
    total_data.columns = ['total_'+i for i in total_data.columns] # 修改列名
    
    return pd.concat([info,total_data,today_data],axis=1) # info、today和total横向合并最终得到汇总的数据

# 定义保存数据方法
def save_data(data,name): 
    file_name = name+'_'+time.strftime('%Y_%m_%d',time.localtime(time.time()))+'.csv'
    data.to_csv(file_name,index=None,encoding='utf_8_sig')
    print(file_name+' 保存成功！')
time.strftime('%Y_%m_%d',time.localtime(time.time()))

'2021_01_07'

In [5]:
areaTree = data['areaTree'] # 取出 areaTree——世界各地实时数据
areaTree[0] # 查看第一个国家的数据

{'today': {'confirm': 2820,
  'suspect': None,
  'heal': 1275,
  'dead': 70,
  'severe': None,
  'storeConfirm': None},
 'total': {'confirm': 149881,
  'suspect': 0,
  'heal': 111599,
  'dead': 5004,
  'severe': 0,
  'input': 0},
 'extData': {},
 'name': '突尼斯',
 'id': '9577772',
 'lastUpdateTime': '2021-01-07 08:23:02',
 'children': []}

In [6]:
for i in range(len(areaTree)):  # 查看各国家名称和更新时间
    print(areaTree[i]['name'],areaTree[i]['lastUpdateTime'])
    if i == 5:
        break

突尼斯 2021-01-07 08:23:02
塞尔维亚 2021-01-07 08:23:02
中国 2021-01-07 08:22:58
日本本土 2021-01-07 08:23:02
泰国 2021-01-07 08:23:01
新加坡 2021-01-07 08:23:02


In [7]:
today_world = get_data(areaTree,['id','lastUpdateTime','name'])
today_world.head()   # 查看获取数据的前5行

,id,lastUpdateTime,name,total_confirm,total_dead,total_heal,total_input,total_severe,total_suspect,today_confirm,today_dead,today_heal,today_input,today_severe,today_storeConfirm,today_suspect
0,9577772,2021-01-07 08:23:02,突尼斯,149881,5004,111599,0.0,0,0,2820.0,70.0,1275.0,NaN,NaN,NaN,NaN
1,9507896,2021-01-07 08:23:02,塞尔维亚,352120,3444,0,0.0,0,0,2882.0,0.0,0.0,NaN,NaN,NaN,NaN
2,0,2021-01-07 08:22:58,中国,97217,4795,91106,4359.0,0,0,90.0,1.0,98.0,11.0,NaN,-9.0,NaN
3,1,2021-01-07 08:23:02,日本本土,259495,3609,206883,0.0,0,0,6009.0,0.0,0.0,NaN,NaN,NaN,NaN
4,2,2021-01-07 08:23:01,泰国,9331,66,4418,0.0,0,0,365.0,0.0,0.0,NaN,NaN,NaN,NaN


In [8]:
save_data(today_world,'today_world')

today_world_2021_01_07.csv 保存成功！


# 世界各国历史数据获取 alltime_world

In [9]:
today_world[['id','name']].head()
country_dict = {key:value for key,value in zip(today_world['id'], today_world['name'])}

# 查看前五个内容
count = 0
for i in country_dict:
    print(i,country_dict[i])
    count += 1
    if count == 5:
        break

9577772 突尼斯
9507896 塞尔维亚
0 中国
1 日本本土
2 泰国


In [10]:
start = time.time()
for country_id in country_dict: # 遍历每个国家的编号
    
    try:
        # 按照编号访问每个国家的数据地址，并获取json数据
        url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode='+country_id
        r = requests.get(url, headers=headers)
        json_data = json.loads(r.text)
        
        # 生成每个国家的数据
        country_data = get_data(json_data['data']['list'],['date'])
        country_data['name'] = country_dict[country_id]

        # 数据叠加
        if country_id == '9577772':
            alltime_world = country_data
        else:
            alltime_world = pd.concat([alltime_world,country_data])
            
        print('-'*20,country_dict[country_id],'成功',country_data.shape,alltime_world.shape,
              ',累计耗时:',round(time.time()-start),'-'*20)
        
        time.sleep(10)

    except:
        print('-'*20,country_dict[country_id],'wrong','-'*20)

-------------------- 突尼斯 成功 (293, 15) (293, 15) ,累计耗时: 0 --------------------
-------------------- 塞尔维亚 成功 (298, 15) (591, 15) ,累计耗时: 10 --------------------
-------------------- 中国 成功 (335, 15) (926, 15) ,累计耗时: 21 --------------------
-------------------- 日本本土 成功 (312, 15) (1238, 15) ,累计耗时: 31 --------------------
-------------------- 泰国 成功 (309, 15) (1547, 15) ,累计耗时: 41 --------------------
-------------------- 新加坡 成功 (326, 15) (1873, 15) ,累计耗时: 51 --------------------
-------------------- 韩国 成功 (349, 15) (2222, 15) ,累计耗时: 61 --------------------
-------------------- 澳大利亚 成功 (308, 15) (2530, 15) ,累计耗时: 72 --------------------
-------------------- 德国 成功 (315, 15) (2845, 15) ,累计耗时: 82 --------------------
-------------------- 美国 成功 (320, 15) (3165, 15) ,累计耗时: 92 --------------------
-------------------- 马来西亚 成功 (312, 15) (3477, 15) ,累计耗时: 102 --------------------
-------------------- 越南 成功 (283, 15) (3760, 15) ,累计耗时: 112 --------------------
-------------------- 圣巴泰勒米 成功 (30, 15) (3790

-------------------- 阿根廷 成功 (299, 15) (23773, 15) ,累计耗时: 1031 --------------------
-------------------- 法属波利尼西亚 成功 (36, 15) (23809, 15) ,累计耗时: 1041 --------------------
-------------------- 巴林 成功 (304, 15) (24113, 15) ,累计耗时: 1051 --------------------
-------------------- 莫桑比克 成功 (266, 15) (24379, 15) ,累计耗时: 1061 --------------------
-------------------- 喀麦隆 成功 (280, 15) (24659, 15) ,累计耗时: 1072 --------------------
-------------------- 乌干达 成功 (277, 15) (24936, 15) ,累计耗时: 1082 --------------------
-------------------- 厄立特里亚 成功 (50, 15) (24986, 15) ,累计耗时: 1092 --------------------
-------------------- 刚果（布） 成功 (261, 15) (25247, 15) ,累计耗时: 1102 --------------------
-------------------- 津巴布韦 成功 (265, 15) (25512, 15) ,累计耗时: 1112 --------------------
-------------------- 丹麦 成功 (309, 15) (25821, 15) ,累计耗时: 1123 --------------------
-------------------- 阿鲁巴 成功 (34, 15) (25855, 15) ,累计耗时: 1133 --------------------
-------------------- 斐济 成功 (244, 15) (26099, 15) ,累计耗时: 1143 --------------------


-------------------- 安道尔 成功 (278, 15) (45262, 15) ,累计耗时: 2152 --------------------
-------------------- 牙买加 成功 (262, 15) (45524, 15) ,累计耗时: 2163 --------------------
-------------------- 格恩西岛 成功 (29, 15) (45553, 15) ,累计耗时: 2173 --------------------
-------------------- 罗马尼亚 成功 (302, 15) (45855, 15) ,累计耗时: 2183 --------------------
-------------------- 阿曼 成功 (303, 15) (46158, 15) ,累计耗时: 2193 --------------------
-------------------- 列支敦士登 成功 (48, 15) (46206, 15) ,累计耗时: 2203 --------------------
-------------------- 马达加斯加 成功 (266, 15) (46472, 15) ,累计耗时: 2214 --------------------


In [11]:
save_data(alltime_world,'alltime_world')

alltime_world_2021_01_07.csv 保存成功！


# 数据预处理

In [18]:
#读取数据疫情数据alltime_world_2020_05_31.csv
import pandas as pd
alltime_world = pd.read_csv('alltime_world_2021_01_07.csv', encoding='utf-8')   # 1）读文件
print( alltime_world.info() )     # 2）查看基本信息
name_dict = {'date':'日期','name':'名称','id':'编号','lastUpdateTime':'更新时间',
             'today_confirm':'当日新增确诊','today_suspect':'当日新增疑似',
             'today_heal':'当日新增治愈','today_dead':'当日新增死亡',
             'today_severe':'当日新增重症','today_storeConfirm':'当日现存确诊', 'today_input':'当日输入',
             'total_confirm':'累计确诊','total_suspect':'累计疑似','total_heal':'累计治愈',
             'total_dead':'累计死亡','total_severe':'累计重症','total_input':'累计输入'}   # 3）创建中文列名字典
alltime_world.rename(columns=name_dict, inplace=True)   # 4)将英文列名更改中文列名

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46472 entries, 0 to 46471
Data columns (total 15 columns):
date                  46472 non-null object
total_confirm         46472 non-null int64
total_dead            46472 non-null int64
total_heal            46472 non-null int64
total_input           46472 non-null int64
total_severe          46472 non-null int64
total_suspect         46472 non-null int64
today_confirm         46472 non-null int64
today_dead            46472 non-null int64
today_heal            46472 non-null int64
today_input           46276 non-null float64
today_severe          12509 non-null float64
today_storeConfirm    0 non-null float64
today_suspect         12909 non-null float64
name                  46472 non-null object
dtypes: float64(4), int64(9), object(2)
memory usage: 5.3+ MB
None


In [19]:
## 使用下面属性和函数查看数据的大概信息
alltime_world.info()   # 查看二维数据表的大信息
print('\n\n查看二维数据表的统计消息，统计数值型数据: \n', alltime_world.describe() )
print('\n\n查看二维数据表的列标签：\n', alltime_world.columns )
print('\n\n查看二维数据表的行标签：\n', alltime_world.index )
print('\n\n查看二维数据表的值：\n', alltime_world.values )
print('\n\n查看二维数据表的前5行：\n', alltime_world.head() )
print('\n\n查看二维数据表的后5行：\n', alltime_world.tail() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46472 entries, 0 to 46471
Data columns (total 15 columns):
日期        46472 non-null object
累计确诊      46472 non-null int64
累计死亡      46472 non-null int64
累计治愈      46472 non-null int64
累计输入      46472 non-null int64
累计重症      46472 non-null int64
累计疑似      46472 non-null int64
当日新增确诊    46472 non-null int64
当日新增死亡    46472 non-null int64
当日新增治愈    46472 non-null int64
当日输入      46276 non-null float64
当日新增重症    12509 non-null float64
当日现存确诊    0 non-null float64
当日新增疑似    12909 non-null float64
名称        46472 non-null object
dtypes: float64(4), int64(9), object(2)
memory usage: 5.3+ MB


查看二维数据表的统计消息，统计数值型数据: 
                累计确诊           累计死亡          累计治愈          累计输入          累计重症  \
count  4.647200e+04   46472.000000  4.647200e+04  46472.000000  46472.000000   
mean   1.739744e+05    4996.287657  1.193124e+05     15.413905      2.380810   
std    8.830433e+05   20634.889600  6.413046e+05    209.917572    103.685836   
min    0.000

In [20]:
# 计算缺失值比例
alltime_world_nan = alltime_world.isnull().sum()/len(alltime_world)
# 转变为百分数
alltime_world_nan.apply(lambda x: format(x, '.1%'))

日期          0.0%
累计确诊        0.0%
累计死亡        0.0%
累计治愈        0.0%
累计输入        0.0%
累计重症        0.0%
累计疑似        0.0%
当日新增确诊      0.0%
当日新增死亡      0.0%
当日新增治愈      0.0%
当日输入        0.4%
当日新增重症     73.1%
当日现存确诊    100.0%
当日新增疑似     72.2%
名称          0.0%
dtype: object

In [21]:
# 先使用线性插值的方式填充缺失值
alltime_world.interpolate(method='linear', inplace=True)

# 再使用0填充缺失值
alltime_world.fillna(0, inplace=True)

# 计算当日现存确诊人数（当日现存确诊=累计确诊-累计治愈-累计死亡）
alltime_world['当日现存确诊'] = alltime_world['累计确诊']-alltime_world['累计治愈']-alltime_world['累计死亡']

# 将日期改成datetime格式
alltime_world['日期'] = pd.to_datetime(alltime_world['日期'])

alltime_world.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46472 entries, 0 to 46471
Data columns (total 15 columns):
日期        46472 non-null datetime64[ns]
累计确诊      46472 non-null int64
累计死亡      46472 non-null int64
累计治愈      46472 non-null int64
累计输入      46472 non-null int64
累计重症      46472 non-null int64
累计疑似      46472 non-null int64
当日新增确诊    46472 non-null int64
当日新增死亡    46472 non-null int64
当日新增治愈    46472 non-null int64
当日输入      46472 non-null float64
当日新增重症    46472 non-null float64
当日现存确诊    46472 non-null int64
当日新增疑似    46472 non-null float64
名称        46472 non-null object
dtypes: datetime64[ns](1), float64(3), int64(10), object(1)
memory usage: 5.3+ MB
